Since the datashare and macro factor data files can be downloaded directly from the download link provided by the paper's authors, the only files we need to download manually are the monthly stock-level data from CRSP, which includes price, earnings, and number of outstanding shares.

In [11]:
import os
import time
from typing import List
import pandas as pd
from tqdm import tqdm
import wrds

`permno.list` exports all permnos from the datashare. This function reads all permnos from the list to tell the program which stock data to download from CRSP.

In [12]:
def load_permnos(filepath: str) -> List[int]:
    """Load permno list from a text file."""
    with open(filepath, "r") as f:
        permnos = [int(p.strip()) for p in f.readlines()]
    print(f"✅ Loaded {len(permnos)} permnos from {filepath}")
    return permnos

Because the number of permno values ​​to be obtained is too large, the data is obtained in blocks.

In [13]:
def chunk_list(data: List[int], chunk_size: int) -> List[List[int]]:
    """Split a list into evenly sized chunks."""
    return [data[i:i + chunk_size] for i in range(0, len(data), chunk_size)]

Next, obtain the required data terms according to the assignment requirements.

In [14]:
def download_crsp_data(db, permnos: List[int], start_date: str, end_date: str, chunk_size: int = 300) -> pd.DataFrame:
    """Download CRSP monthly data for given permnos from WRDS."""
    chunks = chunk_list(permnos, chunk_size)
    print(f"Total permnos: {len(permnos)}, total chunks: {len(chunks)}")

    sql = """
        SELECT 
            a.permno,
            a.date,
            a.ret,
            b.dlret,
            a.shrout,
            a.prc,
            a.vol,
            a.hexcd AS exchcd
        FROM crsp.msf a
        LEFT JOIN crsp.msedelist b
            ON a.permno = b.permno
           AND a.date = b.dlstdt
        WHERE a.date BETWEEN %(start)s AND %(end)s
          AND a.permno = ANY(%(permnos)s::int[])
    """

    df_all = []

    for i, chunk in enumerate(tqdm(chunks, desc="📥 Downloading chunks")):
        for attempt in range(5):
            try:
                df_chunk = db.raw_sql(sql, params={"start": start_date, "end": end_date, "permnos": chunk})
                df_all.append(df_chunk)
                total_rows = sum(len(x) for x in df_all)
                print(f"✅ Chunk {i + 1}/{len(chunks)} downloaded. Total rows so far: {total_rows}")
                break
            except Exception as e:
                print(f"⚠️ Retry {attempt + 1}/5 for chunk {i + 1}: {e}")
                time.sleep(5)
        else:
            print(f"❌ Failed to download chunk {i + 1} after 5 attempts.")

    df = pd.concat(df_all, ignore_index=True)
    df = df.sort_values(["permno", "date"]).reset_index(drop=True)
    return df

Clean up any missing or non-numerical yields in the original WRDS data, calculate the correct adjusted yields, and securely save them as a CSV file.

In [15]:
def clean_and_save(df: pd.DataFrame, output_path: str) -> None:
    """Clean data (compute adjusted returns) and save to CSV."""
    df["ret"] = pd.to_numeric(df["ret"], errors="coerce")
    df["dlret"] = pd.to_numeric(df["dlret"], errors="coerce")
    df["ret_adj"] = (1 + df["ret"].fillna(0)) * (1 + df["dlret"].fillna(0)) - 1

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df.to_csv(output_path, index=False)
    print(f"✅ Data saved to {output_path} | Total rows: {df.shape[0]}")

Next, the main function is defined and executed.

In [17]:
def main():
    """Main execution function."""
    # === Configuration ===
    permno_file = "../data/raw/permno_list.txt"
    output_csv = "../data/raw/crsp_monthly_1957_2021.csv"
    start_date = "1957-01-01"
    end_date = "2021-12-31"

    # === Load data ===
    permnos = load_permnos(permno_file)

    # === Connect to WRDS ===
    print("🔗 Connecting to WRDS...")
    db = wrds.Connection(wrds_username="colinzen")

    # === Download & Clean ===
    df = download_crsp_data(db, permnos, start_date, end_date, chunk_size=300)
    clean_and_save(df, output_csv)

    print("🎉 All done!")


if __name__ == "__main__":
    main()

✅ Loaded 32793 permnos from ../data/raw/permno_list.txt
🔗 Connecting to WRDS...
Loading library list...
Done
Total permnos: 32793, total chunks: 110


📥 Downloading chunks:   1%|▌                                                          | 1/110 [00:21<39:48, 21.91s/it]

✅ Chunk 1/110 downloaded. Total rows so far: 46092


📥 Downloading chunks:   2%|█                                                          | 2/110 [00:53<50:08, 27.85s/it]

✅ Chunk 2/110 downloaded. Total rows so far: 84945


...

📥 Downloading chunks:  99%|█████████████████████████████████████████████████████▌| 109/110 [6:44:19<04:09, 249.52s/it]

✅ Chunk 109/110 downloaded. Total rows so far: 4344068


📥 Downloading chunks: 100%|██████████████████████████████████████████████████████| 110/110 [6:45:26<00:00, 221.15s/it]

✅ Chunk 110/110 downloaded. Total rows so far: 4353483


✅ Data saved to ../data/raw/crsp_monthly_1957_2021.csv | Total rows: 4353483
🎉 All done!
